In [2]:
import os
print("Current Working Directory:", os.getcwd())  # Prints the directory where Python is looking for the file
print("Files in Directory:", os.listdir())  # Lists all files in the current directory


Current Working Directory: c:\Users\vudut\OneDrive\Desktop\Python\MINI Project
Files in Directory: ['.git', 'CNN.ipynb', 'Data Sets', 'FINAL_1.ipynb', 'logs', 'particle_images', 'PPT & Images', 'preprocessed_data.csv', 'README.md', 'Simulation.ipynb', 'wandb', 'xgboost_model.onnx', 'xgboost_optimized.onnx']


In [3]:
import pandas as pd

# Load the preprocessed dataset
df_combined = pd.read_csv("preprocessed_data.csv")

# Verify the data
print(df_combined.head())  # Check the first few rows
print(f"Dataset shape: {df_combined.shape}")  # Print dataset size


     lep_pt   lep_eta   lep_phi     lep_E  lep_charge  lep_type  jet_n  \
0 -0.646637 -0.958065  0.381154 -0.477948         1.0      11.0      1   
1  0.377964 -0.059720  0.108998 -0.612058         0.0      11.0      0   
2  0.132456  0.360554 -0.531584 -0.627914         1.0      11.0      1   
3 -0.646720  0.312437  0.869416 -0.844918        -1.0      11.0      0   
4 -0.261201  0.822651 -0.918316 -0.429899         1.0      11.0      1   

     jet_pt    met_et   met_phi  
0 -0.331582 -1.246566  0.771644  
1       NaN -0.872589 -2.589168  
2 -0.722571  0.418665 -0.035447  
3 -0.859866 -0.008696  1.533058  
4  1.252879  1.282319  2.545199  
Dataset shape: (14945674, 10)


In [ ]:
import tensorflow as tf

# Check available GPUs (should list DirectML GPU)
print("Available Devices:", tf.config.list_physical_devices())

# Force TensorFlow to use DirectML
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[1], True)
print("Using TensorFlow DirectML with GPU")


Available Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Using TensorFlow DirectML with GPU


In [7]:
import pandas as pd

# Load the preprocessed dataset
df_combined = pd.read_csv("preprocessed_data.csv")

# Verify data
print(df_combined.head())  # Check the first few rows


     lep_pt   lep_eta   lep_phi     lep_E  lep_charge  lep_type  jet_n  \
0 -0.646637 -0.958065  0.381154 -0.477948         1.0      11.0      1   
1  0.377964 -0.059720  0.108998 -0.612058         0.0      11.0      0   
2  0.132456  0.360554 -0.531584 -0.627914         1.0      11.0      1   
3 -0.646720  0.312437  0.869416 -0.844918        -1.0      11.0      0   
4 -0.261201  0.822651 -0.918316 -0.429899         1.0      11.0      1   

     jet_pt    met_et   met_phi  
0 -0.331582 -1.246566  0.771644  
1       NaN -0.872589 -2.589168  
2 -0.722571  0.418665 -0.035447  
3 -0.859866 -0.008696  1.533058  
4  1.252879  1.282319  2.545199  


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from multiprocessing import Pool

# Create a directory to store generated images
os.makedirs("particle_images", exist_ok=True)

# Load the preprocessed dataset
df_combined = pd.read_csv("preprocessed_data.csv")

# Convert Pandas DataFrame to NumPy arrays (DirectML-compatible)
lep_eta = df_combined["lep_eta"].values.astype(np.float32)
lep_phi = df_combined["lep_phi"].values.astype(np.float32)
lep_pt = df_combined["lep_pt"].values.astype(np.float32)

# Function to generate images
def generate_image(args):
    idx, eta, phi, pt = args
    fig, ax = plt.subplots(figsize=(2, 2))
    scatter = ax.scatter(eta, phi, c=pt, cmap="inferno")

    ax.set_xlim(-3, 3)
    ax.set_ylim(-np.pi, np.pi)
    ax.axis("off")

    plt.savefig(f"particle_images/{idx}.png", bbox_inches="tight", pad_inches=0)
    plt.close()

# **Use multiprocessing to speed up image generation**
if __name__ == '__main__':
    with Pool(processes=os.cpu_count()) as pool:
        pool.map(generate_image, zip(range(len(lep_eta)), lep_eta, lep_phi, lep_pt))

print("✅ Image generation completed successfully!")


In [ ]:
import os

# Set necessary environment variables
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
# Disable eager execution to avoid early initialization
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# Only then import tensorflow
import tensorflow as tf

# Make sure TF doesn't eagerly initialize the context
tf.compat.v1.disable_eager_execution()

# Configure memory growth
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    print(f"Found GPU devices: {physical_devices}")
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            print(f"Memory growth enabled for {device}")
        except Exception as e:
            print(f"Error setting memory growth: {e}")

# Your remaining imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load preprocessed dataset
df_combined = pd.read_csv("preprocessed_data.csv")

# Reduce dataset: Sample 50,000 images
df_sampled = df_combined.sample(n=50000, random_state=42)

# Convert to NumPy arrays - don't use TensorFlow tensors for the matplotlib workflow
lep_eta = df_sampled["lep_eta"].values
lep_phi = df_sampled["lep_phi"].values
lep_pt = df_sampled["lep_pt"].values

# Create directory for images
os.makedirs("particle_images", exist_ok=True)

# Batch size for processing
batch_size = 100

# Process in batches without TensorFlow
for i in range(0, len(lep_eta), batch_size):
    end_idx = min(i + batch_size, len(lep_eta))
    
    for j in range(i, end_idx):
        # Generate and save image using matplotlib directly
        fig, ax = plt.subplots(figsize=(2, 2))
        scatter = ax.scatter(lep_eta[j], lep_phi[j], c=lep_pt[j], cmap="inferno")
        
        ax.set_xlim(-3, 3)
        ax.set_ylim(-np.pi, np.pi)
        ax.axis("off")
        
        plt.savefig(f"particle_images/{j}.png", bbox_inches="tight", pad_inches=0)
        plt.close()
    
    print(f"Processed images {i} to {end_idx-1}")

print("✅ Image generation completed!")

Found GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Error setting memory growth: Physical devices cannot be modified after being initialized
Error setting memory growth: Physical devices cannot be modified after being initialized
Processed images 0 to 99
Processed images 100 to 199
Processed images 200 to 299
Processed images 300 to 399
Processed images 400 to 499
Processed images 500 to 599
Processed images 600 to 699
Processed images 700 to 799
Processed images 800 to 899
Processed images 900 to 999
Processed images 1000 to 1099
Processed images 1100 to 1199
Processed images 1200 to 1299
Processed images 1300 to 1399
Processed images 1400 to 1499
Processed images 1500 to 1599
Processed images 1600 to 1699
Processed images 1700 to 1799
Processed images 1800 to 1899
